In [ ]:
from google.colab import drive
import sys


drive.mount('/content/drive', force_remount=True)

sys.path.append('/content/drive/MyDrive/dnn_model_optimization')

Mounted at /content/drive


In [ ]:
from utils.data import load_data

((imgs, abits), labels), alphabet = load_data('/content/drive/MyDrive/dnn_model_optimization/data', split=False, blank_idx=-1)

In [ ]:
import tensorflow as tf
from utils.tf_helpers import CER, CTCLoss, warmup_tf_model

model = tf.keras.models.load_model('/content/drive/MyDrive/dnn_model_optimization/weights/crnn_common_fields.h5',
                                   custom_objects={'CTCLoss': CTCLoss, 'CER': CER})

model = warmup_tf_model(model, [(128, 50, 2), (128, 32, 400)])

In [ ]:
import time
from tqdm import trange
import gc

NRUNS = 10
times = list()
BATCH_SIZE = 128

start = time.time()

for i in trange(NRUNS):
  gc.collect()
  y_pred = model.predict([abits, imgs], batch_size=BATCH_SIZE, verbose=0)

print(f'Time spent: {(round((time.time()-start) / NRUNS / BATCH_SIZE, 6))}')

loss = CTCLoss(labels, y_pred)
cer = CER()
cer.update_state(labels, y_pred)

print(f'loss: {round(tf.reduce_mean(loss).numpy(), 6)}, Character Error Rate: {round(cer.result().numpy(), 6)}')

100%|██████████| 10/10 [01:08<00:00,  6.87s/it]


Time spent: 0.053708
loss: 0.2938440144062042, Character Error Rate: 0.0022150001022964716


In [ ]:
from google.colab import runtime

runtime.unassign()